In [1]:
import os,sys
import os,sys
sys.path.append("../../")
from analysis_common import *

In [2]:
from root_common import *

Welcome to JupyROOT 6.24/06


In [3]:
#make a canvas we can use by default
c = ROOT.TCanvas()

In [4]:
#some useful functions
from analysis_helpers import *

In [5]:
#makes plotting more interactive in notebook
%jsroot on

In [6]:
#some useful early definitions
lep_accept_pt = 0.4 #GeV
lep_accept_angle = radians(40)

hadron_accept_angle = radians(40)
hadron_accept_ke = 0.060 #GeV

In [7]:
#NOTE YOU WILL NEED TO MODIFY THIS PARENT DIRECTORY!
dir_files_GEM21_11b = "/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_GEM21_11b_00"
dir_files_G18_02a = "/Users/wketchum/Data/LDMX/eN_Ti_GENIE_v3_2_0/ldmx_eTi_4GeV_G18_02a_00"

In [8]:
#create a list of the files that we have
gst_files_GEM21_11b = glob.glob(f"{dir_files_GEM21_11b}/*gst.root")
gst_files_G18_02a = glob.glob(f"{dir_files_G18_02a}/*gst.root")

In [9]:
#defined in analysis_helpers.py ... creates a TChain from our list of files
gst_chain_GEM21_11b = create_gst_chain(gst_files_GEM21_11b,verbose=True)

Created gst chain from 0 files with 0 total events.


In [10]:
gst_chain_G18_02a = create_gst_chain(gst_files_G18_02a,verbose=True)

Created gst chain from 0 files with 0 total events.


In [11]:
#creates a dataframe from our TChain
# see https://root.cern/doc/master/classROOT_1_1RDataFrame.html for examples
df_gst_GEM21_11b_all = ROOT.RDataFrame(gst_chain_GEM21_11b)

In [12]:
#restrict to just 1 million events
df_gst_GEM21_11b_all = df_gst_GEM21_11b_all.Range(1000000)

In [13]:
#define a function that makes new lepton variables
def define_df_gst_lep_vars(df_gst):
    df_gst = df_gst.Define("ptl","sqrt(pxl*pxl+pyl*pyl)") #pt lepton
    df_gst = df_gst.Define("thetazl","atan2(ptl,pzl)") #theta_z of lepton
    df_gst = df_gst.Define("energy_transfer","Ev-El")
    return df_gst

In [14]:
#apply definitions for new lepton variables
df_gst_GEM21_11b_all = define_df_gst_lep_vars(df_gst_GEM21_11b_all)

runtime_error: Template method resolution failed:
  ROOT::RDF::RInterface<ROOT::Detail::RDF::RRange<ROOT::Detail::RDF::RLoopManager>,void> ROOT::RDF::RInterface<ROOT::Detail::RDF::RRange<ROOT::Detail::RDF::RLoopManager>,void>::Define(experimental::basic_string_view<char,char_traits<char> > name, experimental::basic_string_view<char,char_traits<char> > expression) =>
    runtime_error: GetBranchNames: error in opening the tree gst
  ROOT::RDF::RInterface<ROOT::Detail::RDF::RRange<ROOT::Detail::RDF::RLoopManager>,void> ROOT::RDF::RInterface<ROOT::Detail::RDF::RRange<ROOT::Detail::RDF::RLoopManager>,void>::Define(experimental::basic_string_view<char,char_traits<char> > name, experimental::basic_string_view<char,char_traits<char> > expression) =>
    runtime_error: GetBranchNames: error in opening the tree gst

In [ ]:
#define a function to make a bunch of hadron related variables
#sfx is the suffix on the vars (initial, final)
def define_df_gst_hadron_vars(df_gst,sfx=["i","f"]):
    
    for s in sfx:
        df_gst = df_gst.Define(f"thetaxz{s}",f"atan2(px{s},pz{s})")
        df_gst = df_gst.Define(f"thetayz{s}",f"atan2(py{s},pz{s})")
        df_gst = df_gst.Define(f"pt{s}",f"sqrt(px{s}*px{s}+py{s}*py{s})")
        
        #note, only for 'i' is total momentum not already in the gst tree
        if s=="i":
            df_gst = df_gst.Define(f"p{s}",f"sqrt(px{s}*px{s}+py{s}+py{s}+pz{s}*pz{s})")
            
        df_gst = df_gst.Define(f"thetaz{s}",f"atan2(pt{s},pz{s})")
        df_gst = df_gst.Define(f"mass{s}",f"sqrt(E{s}*E{s}-p{s}*p{s})")
        df_gst = df_gst.Define(f"ke{s}",f"E{s}-mass{s}")
        
        #for the hadrons, get the indices sorted by KE
        df_gst = df_gst.Define(f"idx_ke{s}",f"Reverse(Argsort(ke{s}))")
                
    return df_gst


In [15]:
df_gst_GEM21_11b_all = define_df_gst_hadron_vars(df_gst_GEM21_11b_all)

NameError: name 'define_df_gst_hadron_vars' is not defined

In [ ]:
#can print all of our columns here
for n in df_gst_GEM21_11b_all.GetColumnNames():
    print(n)

In [ ]:
#make a quick check to see that the proton indices are filled, and sorted by KE
rows=10
disp = df_gst_GEM21_11b_all.Range(rows).Define("kef_proton","kef[pdgf==2212]").Display(["pdgf","massf","kef","idx_kef","kef_proton"],rows)
disp.Print()

In [ ]:
#define bins for a 2D plot in log-log space
etransfer_bins = array.array('d',np.logspace(np.log10(0.05),np.log10(4),100).tolist())
q2transfer_bins = array.array('d',np.logspace(np.log10(0.02),np.log10(10),100).tolist())

In [ ]:
#create Q2 versus omega plot
h_lep_q2vw = df_gst_GEM21_11b_all.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,len(q2transfer_bins)-1,q2transfer_bins),
                                          "energy_transfer","Q2")

In [ ]:
h_lep_q2vw.SetTitle("All events;Energy Transfer (GeV);Q^{2} (GeV/c)^{2}")
h_lep_q2vw.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [ ]:
#now, make a filter based on acceptance
df_gst_GEM21_11b_accept = df_gst_GEM21_11b_all.Filter(f"ptl>{lep_accept_pt} && abs(thetazl)<{lep_accept_angle}")

In [ ]:
h_lep_q2vw_accept = df_gst_GEM21_11b_accept.Histo2D(("","",len(etransfer_bins)-1,etransfer_bins,
                                            len(q2transfer_bins)-1,q2transfer_bins),
                                      "energy_transfer","Q2")

In [ ]:
h_lep_q2vw_accept.SetTitle("Events passing electron acceptance;Energy Transfer (GeV);Q^{2} (GeV/c)^{2}")
h_lep_q2vw_accept.Draw("colz")
c.SetLogx()
c.SetLogy()
c.Draw()

In [ ]:
#reset canvas to linear scale
c.SetLogy(0)
c.SetLogx(0)

In [ ]:
#Make 1D lepton pt histogram for QE, MEC..., but for those passing our lepton acceptance
h_lep_pt_QEL = df_gst_GEM21_11b_accept.Filter("qel==1").Histo1D(("","",100,0,3.0),"ptl")
h_lep_pt_MEC = df_gst_GEM21_11b_accept.Filter("mec==1").Histo1D(("","",100,0,3.0),"ptl")

In [ ]:
h_lep_pt_QEL.SetLineColor(ROOT.kBlue)
h_lep_pt_MEC.SetLineColor(ROOT.kRed)
h_lep_pt_QEL.Draw()
h_lep_pt_MEC.Draw("same")
c.Draw()

In [ ]:
#Make plot of KE for all protons
#we can select the protons based on the pdg code

h_kef_p_all = df_gst_GEM21_11b_all.Define("kef_proton","kef[pdgf==2212]").Histo1D(("","",100,0,1.0),"kef_proton")

In [ ]:
h_kef_p_all.Draw()
c.Draw()

In [ ]:
#Make plot of KE for leading proton

#Now instead of plotting for all protons, we can plot for just the 'leading' (most energetic) proton
h_kef_p_max = df_gst_GEM21_11b_all.Define("kef_proton_max","Max(kef[pdgf==2212])").Histo1D(("","",100,0,1.0),"kef_proton_max")

In [ ]:
h_kef_p_max.Draw()
c.Draw()

In [ ]:
#Make plot of KE for leading proton, but let's remove events where the proton is very low energy (below 60 MeV)

#Now instead of plotting for all protons, we can plot for just the 'leading' (most energetic) proton
h_kef_p_max = df_gst_GEM21_11b_all.Define("kef_proton_max","Max(kef[pdgf==2212])").Filter("kef_proton_max>0.060").Histo1D(("","",100,0,1.0),"kef_proton_max")

In [ ]:
h_kef_p_max.Draw()
c.Draw()